# Conversión de modelos de cuadripolos
<img src="./img/logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
En este documento se presentan algunas de las capacidades que posee el módulo PyTC2 para operar con cuadripolos. Se muestran ejemplos de cómo definir redes y sus modelos Z, Y y $T_{ABCD}$ asociados, como también algunas redes que implementan dichos modelos.

* Funciones de conversión y definición de cuadripolos: [Model_conversion](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/cuadripolos/index.html#pytc2.cuadripolos.Model_conversion), [I2Tabcd_s](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/cuadripolos/index.html#pytc2.cuadripolos.I2Tabcd_s), [S2Tabcd_s](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/cuadripolos/index.html#pytc2.cuadripolos.S2Tabcd_s), [Ts2Tabcd_s](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/cuadripolos/index.html#pytc2.cuadripolos.Ts2Tabcd_s)
* Funciones para presentación de markdown y latex: [print_subtitle](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.print_subtitle), [print_latex](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.print_latex), [a_equal_b_latex_s](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.a_equal_b_latex_s)


### Introducción

A lo largo del curso se presentó una metodología sistemática para arribar a una función transferencia $T(s)$ a partir de restricciones de la función de módulo $\vert T(j\omega) \vert $ o retardo $ \tau(\omega) $. Si bien en primera instancia arribamos a una $T_{LP}(s)$ pasabajos, es posible mediante núcleos de transformación el pasaje a otro tipo de transferencias (pasa-alto, pasabanda, etc).


### Referencias

 [1] Frickey, D.A. *Conversions Between S, 2,Y , h, ABCD, and T Parameters which are Valid for Complex Source and Load Impedances*. IEEE TRANSACTIONS ON MICROWAVE THEORY A N D TECHNIQUES. VOL 42, NO 2. FEBRUARY 1994.


In [4]:
from pytc2.cuadripolos import Model_conversion, I2Tabcd_s, S2Tabcd_s, Ts2Tabcd_s
from pytc2.general import print_latex, print_subtitle, a_equal_b_latex_s
import sympy as sp
import os

y11, y12, y21, y22 = sp.symbols('y11, y12, y21, y22', complex=True)
z11, z12, z21, z22 = sp.symbols('z11, z12, z21, z22', complex=True)
A, B, C, D = sp.symbols('A, B, C, D', complex=True)
Ai, Bi, Ci, Di = sp.symbols('Ai, Bi, Ci, Di', complex=True)
h11, h12, h21, h22 = sp.symbols('h11, h12, h21, h22', complex=True)
g11, g12, g21, g22 = sp.symbols('g11, g12, g21, g22', complex=True)
v1, v2, i1, i2 = sp.symbols('v1, v2, i1, i2', complex=True)

# Parámetros Z (impedancia - circ. abierto)
ZZ = sp.Matrix([[z11, z12], [z21, z22]])
# vars. dependientes
vv = sp.Matrix([[v1], [v2]])
# vars. INdependientes
ii = sp.Matrix([[i1], [i2]])

# Parámetros Y (admitancia - corto circ.)
YY = sp.Matrix([[y11, y12], [y21, y22]])
# vars. dependientes
# ii = sp.Matrix([[i1], [i2]])
# vars. INdependientes
# vv = sp.Matrix([[v1], [v2]])

# Parámetros H (híbridos h)
HH = sp.Matrix([[h11, h12], [h21, h22]])
# vars. dependientes
h_dep = sp.Matrix([[v1], [i2]])
# vars. INdependientes
h_ind = sp.Matrix([[i1], [v2]])

# Parámetros G (híbridos g)
GG = sp.Matrix([[g11, g12], [g21, g22]])
# vars. dependientes
g_dep = sp.Matrix([[i1], [v2]])
# vars. INdependientes
g_ind = sp.Matrix([[v1], [i2]])

# Parámetros Tabcd (Transmisión, ABCD)
TT = sp.Matrix([[A, -B], [C, -D]])
# vars. dependientes
t_dep = sp.Matrix([[v1], [i1]])
# vars. INdependientes.  (Signo negativo de corriente)
t_ind = sp.Matrix([[v2], [i2]])

# Parámetros Tdcba (Transmisión inversos, DCBA)
TTi = sp.Matrix([[Ai, Bi], [-Ci, -Di]])
# vars. dependientes
ti_dep = sp.Matrix([[v2], [i2]])
# vars. INdependientes. (Signo negativo de corriente)
ti_ind = sp.Matrix([[v1], [i1]])


def do_conversion_table(model_dct):
    
    str_table = '$ \\begin{array}{ l ' + ' c'*len(model_dct) + ' }' + os.linesep 

    str_models = ''
    
    for src_model in model_dct:
        str_table +=  ' & ' + src_model['model_name']
        str_models +=  ', ' + src_model['model_name']

    print_subtitle('Tabla de conversión: ' + str_models[2:])
    
    str_table = str_table + ' \\\\ ' + os.linesep
    
    for dst_model in model_dct:
    
        str_table +=   dst_model['model_name']   + ' & '
        
        for src_model in model_dct:
            
            if src_model['model_name'] == dst_model['model_name']:
                
                str_table +=   dst_model['model_name']   + ' & '

            else:
                HH_z = Model_conversion( src_model, dst_model )
                str_table +=  sp.latex( HH_z['matrix'] )  + ' & '
            
        str_table = str_table[:-2] + ' \\\\ ' + os.linesep
        
        #print_latex( str_table )
        
    str_table = str_table[:-4] + os.linesep + '\\end{array} $'
    
    print_latex( str_table )



### Conversión entre parámetros de cuadripolos

Con las funciones para crear la tabla de N x N modelos se procede a su uso, para ello solo basta definir un diccionario con las N definiciones de cada modelo.

In [5]:
# Definimos el diccionario  con la definición de cada modelo
model_dct = [ { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv },
              { 'model_name': 'H', 'matrix': HH, 'dep_var': h_dep, 'indep_var':h_ind },
              { 'model_name': 'G', 'matrix': GG, 'dep_var': g_dep, 'indep_var':g_ind },
              { 'model_name': 'T', 'matrix': TT, 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True },
              { 'model_name': 'Ti', 'matrix': TTi, 'dep_var': ti_dep, 'indep_var':ti_ind, 'neg_i2_current': True}
            ]

do_conversion_table(model_dct)

#### Tabla de conversión: Z, Y, H, G, T, Ti

<IPython.core.display.Math object>

In [6]:

# Parámetros Imagen
gamma, z1, z2 = sp.symbols('\gamma, z1, z2', complex=True)

Tim = I2Tabcd_s(gamma, z1, z2)
 #(Signo negativo de corriente)
Tim[:,1] = Tim[:,1] * -1
# vars. dependientes
tim_dep = t_dep
# vars. INdependientes.
tim_ind = t_ind

# Diccionario con la definición de cada modelo
model_dct = [ { 'model_name': 'Im', 'matrix': Tim, 'dep_var': tim_dep, 'indep_var':tim_ind, 'neg_i2_current': True }, # T_ABCD
              { 'model_name': 'T_{ABCD}', 'matrix': TT, 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True }, # T_ABCD
              { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv }
            ]

do_conversion_table(model_dct)


#### Tabla de conversión: Im, T_{ABCD}, Z, Y

<IPython.core.display.Math object>

In [8]:
s11, s12, s21, s22 = sp.symbols('s11, s12, s21, s22', complex=True)
ts11, ts12, ts21, ts22 = sp.symbols('t11, t12, t21, t22', complex=True)

# ondas normalizadas de tensión
a1, a2, b1, b2 = sp.symbols('a1, a2, b1, b2', complex=True)

# impedancia característica
Z01, Z02 = sp.symbols('Z01, Z02', complex=True)
Zo = sp.symbols('Zo', real=True)

# Parámetros dispersión (scattering - S)
Spar = sp.Matrix([[s11, s12], [s21, s22]])
# vars. dependientes
bb = sp.Matrix([[b1], [b2]])
# vars. INdependientes
aa = sp.Matrix([[a1], [a2]])

# a y b en función de tensiones y corrientes, para vincular con cuadripolos convencionales
aa_iv = aa.subs({a1:(v1 + Z01*i1)/sp.sqrt(2*(Z01 + sp.conjugate(Z01))), a2:(v2 + Z02*i2)/sp.sqrt(2*(Z02 + sp.conjugate(Z02)))})
bb_iv = bb.subs({b1:(v1 - sp.conjugate(Z01)*i1)/sp.sqrt(2*(Z01 + sp.conjugate(Z01))), b2:(v2 - sp.conjugate(Z02)*i2)/sp.sqrt(2*(Z02 + sp.conjugate(Z02)))})

# Parámetros transmisión dispersión (scattering transfer param.)
TSpar = sp.Matrix([[ts11, ts12], [ts21, ts22]])
# vars. dependientes
ts1 = sp.Matrix([[a1], [b1]])
# vars. INdependientes
ts2 = sp.Matrix([[b2], [a2]])

# ts1,2 en función de tensiones y corrientes, para vincular con cuadripolos convencionales
ts1_iv = aa.subs({a1:(v1 + Z01*i1)/sp.sqrt(2*(Z01 + sp.conjugate(Z01))), b1:(v1 - sp.conjugate(Z01)*i1)/sp.sqrt(2*(Z01 + sp.conjugate(Z01)))})
ts2_iv = bb.subs({b2:(v2 - sp.conjugate(Z02)*i2)/sp.sqrt(2*(Z02 + sp.conjugate(Z02))), a2:(v2 + Z02*i2)/sp.sqrt(2*(Z02 + sp.conjugate(Z02)))})

#Tabcd_proxy_model = { 'model_name': 'T_{ABCD}', 'matrix': TT, 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True }

# Diccionario con la definición de cada modelo
model_dct = [ { 'model_name': 'S', 'matrix': Spar, 'dep_var': bb, 'indep_var':aa },
              { 'model_name': 'T_S', 'matrix': TSpar, 'dep_var': ts1, 'indep_var':ts2 },
            ]
do_conversion_table(model_dct)

# Diccionario con la definición de cada modelo:
# En este caso convertimos el modelo S a un ABCD primero, con sus respectivas variables
model_dct = [ { 'model_name': 'S', 'matrix': Spar, 'proxy_matrix': S2Tabcd_s( Spar, sp.Rational(1), sp.Rational(1)), 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True },              
              { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv }
            ]

do_conversion_table(model_dct)

# Diccionario con la definición de cada modelo:
# En este caso convertimos el modelo Ts a un ABCD primero, con sus respectivas variables
model_dct = [ { 'model_name': 'T_S', 'matrix': TSpar, 'proxy_matrix': Ts2Tabcd_s( TSpar, Z01, Z02), 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True },
              { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv }
            ]

do_conversion_table(model_dct)

#### Tabla de conversión: S, T_S

<IPython.core.display.Math object>

#### Tabla de conversión: S, Z, Y

<IPython.core.display.Math object>

#### Tabla de conversión: T_S, Z, Y

<IPython.core.display.Math object>